# LSTMs for Human Activity Recognition

Human activity recognition using smartwatch dataset and an LSTM RNN. Classifying the type of movement amongst six categories:
- WALKING,
- SITTING,
- STANDING,
- LAYING.

Compared to a classical approach, using a Recurrent Neural Networks (RNN) with Long Short-Term Memory cells (LSTMs) require no or almost no feature engineering. Data can be fed directly into the neural network who acts like a black box, modeling the problem correctly. Other research on the activity recognition dataset used mostly use a big amount of feature engineering, which is rather a signal processing approach combined with classical data science techniques. The approach here is rather very simple in terms of how much did the data was preprocessed. 

## Details about input data

I will be using an LSTM on the data to learn (as a smartwatch attached on the wrist) to recognise the type of activity that the user is doing. A similar dataset's description based on prior research goes like this:

> The sensor signals (accelerometer and gyroscope) were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used. 

That said, I will use the almost raw data. 

## What is an RNN?

As explained in [this article](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), an RNN takes many input vectors to process them and output other vectors. It can be roughly pictured like in the image below, imagining each rectangle has a vectorial depth and other special hidden quirks in the image below. **In our case, the "many to one" architecture is used**: we accept time series of feature vectors (one vector per time step) to convert them to a probability vector at the output for classification. Note that a "one to one" architecture would be a standard feedforward neural network. 

<img src="http://karpathy.github.io/assets/rnn/diags.jpeg" />

An LSTM is an improved RNN. It is more complex, but easier to train, avoiding what is called the vanishing gradient problem. 


## Results 

Scroll on! Nice visuals awaits. 

In [32]:
# All Includes

import numpy as np
import pandas
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
from sklearn.utils import shuffle

import os

In [2]:
import glob
import h5py
%matplotlib inline

mat_names = glob.glob('../project_datasets/*.mat')
# each test subject got a different file - 9 test subjects
print(mat_names)

matfile = h5py.File(mat_names[0], 'r')
print(matfile.keys()) #image and type

image_mat = matfile['image']
image_shape = image_mat.shape # 288 (48x6) trials across 25 electrodes for 1000 time points (250Hz*4s)
print image_shape

['../project_datasets/A07T_slice.mat', '../project_datasets/A08T_slice.mat', '../project_datasets/A09T_slice.mat', '../project_datasets/A03T_slice.mat', '../project_datasets/A04T_slice.mat', '../project_datasets/A01T_slice.mat', '../project_datasets/A05T_slice.mat', '../project_datasets/A06T_slice.mat', '../project_datasets/A02T_slice.mat']
[u'image', u'type']
(288, 25, 1000)


In [3]:
type_mat = matfile['type']
type_shape = type_mat.shape
print type_shape
# plt.plot(type_mat[0,:288]) # gets the significant values of types
# all the 0's occur after 288, and are meaningless I think
# so the image_mat, which has shape (288, 25, 1000) should correspond
# to the first 288 entries of type_mat, so
# for a single subject, training data should be image_mat, with 288 samples, each sample has shape (25, 1000)
# and our target label matrix should be type_mat[:288] (or 287?)

nans = np.sum(np.isnan(image_mat[:,:]))
print(nans) #No NaN in the data
print len(image_mat[0:,:])
count = 0
# for i in range(len(image_mat[0:,:])):
#  if np.sum(np.isnan(image_mat[i:,:])):
#         pass

type_set = list(set(type_mat[0,:]))
print(type_set) 

(1, 1000)
0
288
[0.0, 769.0, 770.0, 771.0, 772.0]


In [4]:
EEG_channels = 22 #from project guidelines
test_count = 50 #from project guideline, 238 for train-validation and 50 for test
validation_count = 38 # 38 points in validation set and remaining 200 points in test set

In [5]:
#setting seed
np.random.seed(seed=1337)
test_picked = np.random.choice(image_shape[0], test_count, replace=False)
train_val_picked = np.setdiff1d(np.arange(image_shape[0]), test_picked)
val_picked = train_val_picked[:validation_count]
train_picked = train_val_picked[validation_count:]

In [6]:
trainval_data_X = []
training_data_X = []
validation_data_X = []
test_data_X = []

trainval_data_Y = []
training_data_Y = []
validation_data_Y = []
test_data_Y = []

for i in range(len(mat_names)):
    matfile = h5py.File(mat_names[i], 'r')
    
    trainval_data_X.append(matfile['image'][sorted(train_val_picked),:EEG_channels,:]) #(238, 22, 1000) x 9
    training_data_X.append(matfile['image'][sorted(train_picked),:EEG_channels,:]) #(200, 22, 1000) x 9
    validation_data_X.append(matfile['image'][sorted(val_picked),:EEG_channels,:]) #(38, 22, 1000) x 9
    test_data_X.append(matfile['image'][sorted(test_picked),:EEG_channels,:]) #(50, 22, 1000) x 9
    
    trainval_data_Y.append(matfile['type'][0,sorted(train_val_picked)] - type_set[1]) #(238, ) x 9
    training_data_Y.append(matfile['type'][0,sorted(train_picked)] - type_set[1]) #(200, ) x 9
    validation_data_Y.append(matfile['type'][0,sorted(val_picked)] - type_set[1]) #(38, ) x 9
    test_data_Y.append(matfile['type'][0,sorted(test_picked)] - type_set[1]) #(50, ) x 9

In [7]:
training_data_shape = training_data_X[0].shape
print(training_data_shape) #(200, 22, 1000) while test data shape is (50, 22, 1000) and validation data is (38, 22,1000)

(200, 22, 1000)


In [8]:
from functools import reduce

rnn_trainval_data_X = np.concatenate(trainval_data_X, axis=0) #(2142, 22, 1000)
rnn_training_data_X = np.concatenate(training_data_X, axis=0) #(1800, 22, 1000)
rnn_validation_data_X = np.concatenate(validation_data_X, axis=0) #(342, 22, 1000)
rnn_test_data_X = np.concatenate(test_data_X, axis=0) #(450, 22, 1000)

rnn_trainval_data_Y = np.concatenate(trainval_data_Y, axis=0) #(2142, )
rnn_training_data_Y = np.concatenate(training_data_Y, axis=0) #(1800, )
rnn_validation_data_Y = np.concatenate(validation_data_Y, axis=0) #(342, )
rnn_test_data_Y = np.concatenate(test_data_Y, axis=0) #(450,)

def remove_nan_rows_A(A, b, debug=True):
    if (debug):
        print('before nans: {}'.format(str(A.shape)))
    if (np.isnan(A).any() or np.isnan(b).any()):
        mask = ~np.isnan(np.sum(A,axis=(1,2))) & ~np.isnan(b[:])
        A = A[mask, :, :]
        b = b[mask]
    
    if (debug):
        print('before nans: {}'.format(str(A.shape)))
    assert A.shape[0] == b.shape[0]
    return A, b

rnn_trainval_data_X, rnn_trainval_data_Y = remove_nan_rows_A(rnn_trainval_data_X,
                                                             rnn_trainval_data_Y)
rnn_training_data_X, rnn_training_data_Y = remove_nan_rows_A(rnn_training_data_X, 
                                                             rnn_training_data_Y)
rnn_validation_data_X, rnn_validation_data_Y = remove_nan_rows_A(rnn_validation_data_X,
                                         rnn_validation_data_Y)
rnn_test_data_X, rnn_test_data_Y = remove_nan_rows_A(rnn_test_data_X,
                                   rnn_test_data_Y)

new_axis = (0,2,1)
rnn_trainval_data_X = np.transpose(rnn_trainval_data_X, new_axis)
rnn_training_data_X = np.transpose(rnn_training_data_X, new_axis)
rnn_validation_data_X = np.transpose(rnn_validation_data_X, new_axis)
rnn_test_data_X = np.transpose(rnn_test_data_X, new_axis)

# repeating the Y labels for the rnn
N_trainval, T, E = rnn_trainval_data_X.shape
N_training, _, _ = rnn_trainval_data_X.shape
N_validation, _, _ = rnn_test_data_X.shape
N_test, _, _ = rnn_test_data_X.shape

before nans: (2142, 22, 1000)
before nans: (2115, 22, 1000)
before nans: (1800, 22, 1000)
before nans: (1775, 22, 1000)
before nans: (342, 22, 1000)
before nans: (340, 22, 1000)
before nans: (450, 22, 1000)
before nans: (443, 22, 1000)


In [9]:
training_data_shape = training_data_X[0].shape
print(training_data_shape) #(200, 22, 1000) while test data shape is (50, 22, 1000) and validation data is (38, 22,1000)

(200, 22, 1000)


In [10]:
mean_list = np.mean(rnn_trainval_data_X.reshape(-1, rnn_trainval_data_X.shape[-1]), axis=0)
std_list = np.sqrt((np.var(rnn_trainval_data_X.reshape(-1, rnn_trainval_data_X.shape[-1]), axis=0)))
print(mean_list)
print(std_list)

[0.24211577 0.29200906 0.28058568 0.28058862 0.27744969 0.30503537
 0.37432564 0.35469766 0.29599771 0.28391495 0.28070525 0.32476685
 0.33706837 0.3755972  0.37135214 0.37447009 0.3089289  0.33019716
 0.29176846 0.34339516 0.36118418 0.32799225]
[11.27145177 10.2505745  10.92481975 11.49970902 11.21479773 10.89149331
  8.71352886 10.06750202 10.70168511 11.33973592 10.97697775 10.88472781
 10.10025118 10.33649046 10.64618123 11.09796571 10.98326707 11.08252466
 10.81161869 11.24811363 11.16262363 11.35832584]


In [11]:
rnn_trainval_data_X = (rnn_trainval_data_X - mean_list)/std_list
rnn_training_data_X = (rnn_training_data_X - mean_list)/std_list
rnn_validation_data_X = (rnn_validation_data_X - mean_list)/std_list
rnn_test_data_X = (rnn_test_data_X - mean_list)/std_list

In [12]:
print rnn_trainval_data_X.shape
print rnn_trainval_data_Y.shape

(2115, 1000, 22)
(2115,)


In [13]:
set(rnn_trainval_data_Y)

{0.0, 1.0, 2.0, 3.0}

In [14]:
# Useful Constants

# Those are separate normalised input features for the neural network

# Output classes to learn how to classify
LABELS = [
    "LEFT",
    "RIGHT",
    "FOOT",
    "TONGUE"
    
] 


In [15]:
pandas.DataFrame(rnn_trainval_data_X[3]).describe()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.019228,-0.082219,-0.075826,-0.047379,-0.050532,-0.060178,-0.063592,-0.079086,-0.051713,-0.044552,...,-0.049065,-0.060679,-0.061354,-0.056502,-0.046323,-0.063935,-0.059752,-0.086984,-0.081453,-0.055814
std,0.656299,0.594934,0.653857,0.670126,0.748175,0.684128,0.582003,0.595916,0.659181,0.691749,...,0.662745,0.646819,0.665252,0.665526,0.664200,0.651515,0.680298,0.660292,0.651054,0.725231
min,-1.658983,-1.633771,-1.791123,-1.926622,-1.992703,-2.157500,-2.228408,-1.994662,-1.975914,-1.803387,...,-2.368362,-2.147902,-2.208856,-1.837634,-2.104261,-2.179857,-2.172212,-2.166304,-2.175740,-2.100939
25%,-0.472010,-0.485779,-0.526264,-0.492525,-0.566800,-0.494254,-0.452030,-0.482651,-0.497613,-0.529908,...,-0.502304,-0.489827,-0.490086,-0.482515,-0.481587,-0.480294,-0.501194,-0.508040,-0.500401,-0.544743
50%,-0.066967,-0.128520,-0.132951,-0.088090,-0.113995,-0.106462,-0.087789,-0.107983,-0.082411,-0.076708,...,-0.069630,-0.090661,-0.096798,-0.068940,-0.061470,-0.093679,-0.106021,-0.113008,-0.106719,-0.101958
75%,0.390061,0.277565,0.307292,0.340759,0.368200,0.339611,0.310075,0.304273,0.347620,0.389409,...,0.359418,0.347477,0.336621,0.327036,0.345310,0.341400,0.329798,0.286364,0.286964,0.393489
max,3.006600,2.648573,2.950982,3.338457,3.754447,3.074328,1.839889,2.084250,2.691685,3.032174,...,2.644856,2.377555,2.643603,3.024075,2.692631,2.459520,2.529219,2.539346,2.535329,2.675121


## Additionnal Parameters:

Here are some core parameter definitions for the training. 

The whole neural network's structure could be summarised by enumerating those parameters and the fact an LSTM is used. 

In [16]:
# Input Data 
X_train = rnn_training_data_X
X_test = rnn_test_data_X
X_val = rnn_validation_data_X
y_train = rnn_training_data_Y
y_test = rnn_test_data_Y
y_val = rnn_validation_data_Y

training_data_count = len(X_train)  # 2115 training series 
test_data_count = len(X_test)  # 443 testing series
n_steps = len(X_train[0])  # 1000 timesteps per series
n_input = len(X_train[0][0])  # 22 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = 4 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
dropout = 1.0
lambda_loss_amount = 0.0015
training_iters = training_data_count * 100  # Loop 10 times on the dataset
batch_size = 100
display_iter = 10000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")


Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
((443, 1000, 22), (443,), 0.0016580587469994378, 0.997292271181712)
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


## Utility functions for training:

In [45]:
class Config(object):
    """
    define a class to store parameters,
    the input should be feature mat of training and testing
    """

    def __init__(self, X_train, X_test):
        # Data shaping
        self.train_count = len(X_train)  # 7352 training series
        self.test_data_count = len(X_test)  # 2947 testing series
        self.n_steps = len(X_train[0])  # 128 time_steps per series
        self.n_classes = 4  # Final output classes

        # Training
        self.learning_rate = 0.0025
        self.lambda_loss_amount = 0.005
        self.training_epochs = 250
        self.batch_size = 100
        self.clip_gradients = 15.0
        self.gradient_noise_scale = None
        # Dropout is added on inputs and after each stacked layers (but not
        # between residual layers).
        self.keep_prob_for_dropout = 0.90  # **(1/3.0)

        # Linear+relu structure
        self.bias_mean = 0.3
        # I would recommend between 0.1 and 1.0 or to change and use a xavier
        # initializer
        self.weights_stddev = 0.2

        ########
        # NOTE: I think that if any of the below parameters are changed,
        # the best is to readjust every parameters in the "Training" section
        # above to properly compare the architectures only once optimised.
        ########

        # LSTM structure
        # Features count is of 9: three 3D sensors features over time
        self.n_inputs = len(X_train[0][0])
        self.n_hidden = 10  # nb of neurons inside the neural network
        # Use bidir in every LSTM cell, or not:
        self.use_bidirectionnal_cells = False

        # High-level deep architecture
        self.also_add_dropout_between_stacked_cells = False  # True
        # NOTE: values of exactly 1 (int) for those 2 high-level parameters below totally disables them and result in only 1 starting LSTM.
        # self.n_layers_in_highway = 1  # Number of residual connections to the LSTMs (highway-style), this is did for each stacked block (inside them).
        # self.n_stacked_layers = 1  # Stack multiple blocks of residual
        # layers.

In [46]:
def one_hot(y):
    """convert label from dense to one hot
      argument:
        label: ndarray dense label ,shape: [sample_num,1]
      return:
        one_hot_label: ndarray  one hot, shape: [sample_num,n_class]
    """
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]

    y = y.reshape(len(y))
    n_values = np.max(y) + 1
    return np.eye(n_values)[np.array(y, dtype=np.int32)]  # Returns FLOATS


def batch_norm(input_tensor, config, i):
    # Implementing batch normalisation: this is used out of the residual layers
    # to normalise those output neurons by mean and standard deviation.

    if config.n_layers_in_highway == 0:
        # There is no residual layers, no need for batch_norm:
        return input_tensor

    with tf.variable_scope("batch_norm") as scope:
        if i != 0:
            # Do not create extra variables for each time step
            scope.reuse_variables()

        # Mean and variance normalisation simply crunched over all axes
        axes = list(range(len(input_tensor.get_shape())))

        mean, variance = tf.nn.moments(input_tensor, axes=axes, shift=None, name=None, keep_dims=False)
        stdev = tf.sqrt(variance+0.001)

        # Rescaling
        bn = input_tensor - mean
        bn /= stdev
        # Learnable extra rescaling

        # tf.get_variable("relu_fc_weights", initializer=tf.random_normal(mean=0.0, stddev=0.0)
        bn *= tf.get_variable("a_noreg", initializer=tf.random_normal([1], mean=0.5, stddev=0.0))
        bn += tf.get_variable("b_noreg", initializer=tf.random_normal([1], mean=0.0, stddev=0.0))
        # bn *= tf.Variable(0.5, name=(scope.name + "/a_noreg"))
        # bn += tf.Variable(0.0, name=(scope.name + "/b_noreg"))

    return bn

def relu_fc(input_2D_tensor_list, features_len, new_features_len, config):
    """make a relu fully-connected layer, mainly change the shape of tensor
       both input and output is a list of tensor
        argument:
            input_2D_tensor_list: list shape is [batch_size,feature_num]
            features_len: int the initial features length of input_2D_tensor
            new_feature_len: int the final features length of output_2D_tensor
            config: Config used for weights initializers
        return:
            output_2D_tensor_list lit shape is [batch_size,new_feature_len]
    """

    W = tf.get_variable(
        "relu_fc_weights",
        initializer=tf.random_normal(
            [features_len, new_features_len],
            mean=0.0,
            stddev=float(config.weights_stddev)
        )
    )
    b = tf.get_variable(
        "relu_fc_biases_noreg",
        initializer=tf.random_normal(
            [new_features_len],
            mean=float(config.bias_mean),
            stddev=float(config.weights_stddev)
        )
    )

    # intra-timestep multiplication:
    output_2D_tensor_list = [
        tf.nn.relu(tf.matmul(input_2D_tensor, W) + b)
            for input_2D_tensor in input_2D_tensor_list
    ]

    return output_2D_tensor_list


def single_LSTM_cell(input_hidden_tensor, n_outputs):
    """ define the basic LSTM layer
        argument:
            input_hidden_tensor: list a list of tensor,
                                 shape: time_steps*[batch_size,n_inputs]
            n_outputs: int num of LSTM layer output
        return:
            outputs: list a time_steps list of tensor,
                     shape: time_steps*[batch_size,n_outputs]
    """
    with tf.variable_scope("lstm_cell"):
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_outputs, state_is_tuple=True, forget_bias=0.999)
        outputs, _ = tf.nn.static_rnn(lstm_cell, input_hidden_tensor, dtype=tf.float32)
    return outputs


def bi_LSTM_cell(input_hidden_tensor, n_inputs, n_outputs, config):
    """build bi-LSTM, concatenating the two directions in an inner manner.
        argument:
            input_hidden_tensor: list a time_steps series of tensor, shape: [sample_num, n_inputs]
            n_inputs: int units of input tensor
            n_outputs: int units of output tensor, each bi-LSTM will have half those internal units
            config: Config used for the relu_fc
        return:
            layer_hidden_outputs: list a time_steps series of tensor, shape: [sample_num, n_outputs]
    """
    n_outputs = int(n_outputs/2)

    print "bidir:"

    with tf.variable_scope('pass_forward') as scope2:
        hidden_forward = relu_fc(input_hidden_tensor, n_inputs, n_outputs, config)
        forward = single_LSTM_cell(hidden_forward, n_outputs)

    print (len(hidden_forward), str(hidden_forward[0].get_shape()))

    # Backward pass is as simple as surrounding the cell with a double inversion:
    with tf.variable_scope('pass_backward') as scope2:
        hidden_backward = relu_fc(input_hidden_tensor, n_inputs, n_outputs, config)
        backward = list(reversed(single_LSTM_cell(list(reversed(hidden_backward)), n_outputs)))

    with tf.variable_scope('bidir_concat') as scope:
        # Simply concatenating cells' outputs at each timesteps on the innermost
        # dimension, like if the two cells acted as one cell
        # with twice the n_hidden size:
        layer_hidden_outputs = [
            tf.concat(len(f.get_shape()) - 1, [f, b])
                for f, b in zip(forward, backward)]

    return layer_hidden_outputs


def residual_bidirectional_LSTM_layers(input_hidden_tensor, n_input, n_output, layer_level, config, keep_prob_for_dropout):
    """This architecture is only enabled if "config.n_layers_in_highway" has a
    value only greater than int(0). The arguments are same than for bi_LSTM_cell.
    arguments:
        input_hidden_tensor: list a time_steps series of tensor, shape: [sample_num, n_inputs]
        n_inputs: int units of input tensor
        n_outputs: int units of output tensor, each bi-LSTM will have half those internal units
        config: Config used for determining if there are residual connections and if yes, their number and with some batch_norm.
    return:
        layer_hidden_outputs: list a time_steps series of tensor, shape: [sample_num, n_outputs]
    """
    with tf.variable_scope('layer_{}'.format(layer_level)) as scope:

        if config.use_bidirectionnal_cells:
            get_lstm = lambda input_tensor: bi_LSTM_cell(input_tensor, n_input, n_output, config)
        else:
            get_lstm = lambda input_tensor: single_LSTM_cell(relu_fc(input_tensor, n_input, n_output, config), n_output)
        def add_highway_redisual(layer, residual_minilayer):
            return [a + b for a, b in zip(layer, residual_minilayer)]

        hidden_LSTM_layer = get_lstm(input_hidden_tensor)
        # Adding K new (residual bidir) connections to this first layer:
        for i in range(config.n_layers_in_highway - 1):
            with tf.variable_scope('LSTM_residual_{}'.format(i)) as scope2:
                hidden_LSTM_layer = add_highway_redisual(
                    hidden_LSTM_layer,
                    get_lstm(input_hidden_tensor)
                )

        if config.also_add_dropout_between_stacked_cells:
            hidden_LSTM_layer = [tf.nn.dropout(out, keep_prob_for_dropout) for out in hidden_LSTM_layer]

        return [batch_norm(out, config, i) for i, out in enumerate(hidden_LSTM_layer)]


def LSTM_network(feature_mat, config, keep_prob_for_dropout):
    """model a LSTM Network,
      it stacks 2 LSTM layers, each layer has n_hidden=32 cells
       and 1 output layer, it is a full connet layer
      argument:
        feature_mat: ndarray fature matrix, shape=[batch_size,time_steps,n_inputs]
        config: class containing config of network
      return:
              : ndarray  output shape [batch_size, n_classes]
    """

    with tf.variable_scope('LSTM_network') as scope:  # TensorFlow graph naming

        feature_mat = tf.nn.dropout(feature_mat, keep_prob_for_dropout)

        # Exchange dim 1 and dim 0
        feature_mat = tf.transpose(feature_mat, [1, 0, 2])
        print feature_mat.get_shape()
        # New feature_mat's shape: [time_steps, batch_size, n_inputs]

        # Temporarily crush the feature_mat's dimensions
        feature_mat = tf.reshape(feature_mat, [-1, config.n_inputs])
        print feature_mat.get_shape()
        # New feature_mat's shape: [time_steps*batch_size, n_inputs]

        # Split the series because the rnn cell needs time_steps features, each of shape:
        hidden = tf.split(feature_mat, config.n_steps, 0)
        print (len(hidden), str(hidden[0].get_shape()))
        # New shape: a list of lenght "time_step" containing tensors of shape [batch_size, n_hidden]

        # Stacking LSTM cells, at least one is stacked:
        print "\nCreating hidden #1:"
        hidden = residual_bidirectional_LSTM_layers(hidden, config.n_inputs, config.n_hidden, 1, config, keep_prob_for_dropout)
        print (len(hidden), str(hidden[0].get_shape()))

        for stacked_hidden_index in range(config.n_stacked_layers - 1):
            # If the config permits it, we stack more lstm cells:
            print "\nCreating hidden #{}:".format(stacked_hidden_index+2)
            hidden = residual_bidirectional_LSTM_layers(hidden, config.n_hidden, config.n_hidden, stacked_hidden_index+2, config, keep_prob_for_dropout)
            print (len(hidden), str(hidden[0].get_shape()))

        print ""

        # Final fully-connected activation logits
        # Get the last output tensor of the inner loop output series, of shape [batch_size, n_classes]
        last_hidden = tf.nn.dropout(hidden[-1], keep_prob_for_dropout)
        last_logits = relu_fc(
            [last_hidden],
            config.n_hidden, config.n_classes, config
        )[0]
        return last_logits


def run_with_config(Config, X_train, y_train, X_test, y_test):
    tf.reset_default_graph()  # To enable to run multiple things in a loop

    #-----------------------------------
    # Define parameters for model
    #-----------------------------------
    config = Config(X_train, X_test)
    print("Some useful info to get an insight on dataset's shape and normalisation:")
    print("features shape, labels shape, each features mean, each features standard deviation")
    print(X_test.shape, y_test.shape,
          np.mean(X_test), np.std(X_test))
    print("the dataset is therefore properly normalised, as expected.")

    #------------------------------------------------------
    # Let's get serious and build the neural network
    #------------------------------------------------------
    #with tf.device("/cpu:0"):  # Remove this line to use GPU. If you have a too small GPU, it crashes.
    X = tf.placeholder(tf.float32, [
                       None, config.n_steps, config.n_inputs], name="X")
    Y = tf.placeholder(tf.float32, [
                       None, config.n_classes], name="Y")

    # is_train for dropout control:
    is_train = tf.placeholder(tf.bool, name="is_train")
    keep_prob_for_dropout = tf.cond(is_train,
        lambda: tf.constant(
            config.keep_prob_for_dropout,
            name="keep_prob_for_dropout"
        ),
        lambda: tf.constant(
            1.0,
            name="keep_prob_for_dropout"
        )
    )

    pred_y = LSTM_network(X, config, keep_prob_for_dropout)

    # Loss, optimizer, evaluation

    # Softmax loss with L2 and L1 layer-wise regularisation
    print "Unregularised variables:"
    for unreg in [tf_var.name for tf_var in tf.trainable_variables() if ("noreg" in tf_var.name or "Bias" in tf_var.name)]:
        print unreg
    l2 = config.lambda_loss_amount * sum(
        tf.nn.l2_loss(tf_var)
            for tf_var in tf.trainable_variables()
            if not ("noreg" in tf_var.name or "Bias" in tf_var.name)
    )
    # first_weights = [w for w in tf.all_variables() if w.name == 'LSTM_network/layer_1/pass_forward/relu_fc_weights:0'][0]
    # l1 = config.lambda_loss_amount * tf.reduce_mean(tf.abs(first_weights))
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=pred_y)) + l2  # + l1

    # Gradient clipping Adam optimizer with gradient noise
    optimize = tf.contrib.layers.optimize_loss(
        loss,
        global_step=tf.Variable(0),
        learning_rate=config.learning_rate,
        optimizer=tf.train.AdamOptimizer(learning_rate=config.learning_rate),
        clip_gradients=config.clip_gradients,
        gradient_noise_scale=config.gradient_noise_scale
    )

    correct_pred = tf.equal(tf.argmax(pred_y, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))

    #--------------------------------------------
    # Hooray, now train the neural network
    #--------------------------------------------
    # Note that log_device_placement can be turned of for less console spam.

    sessconfig = tf.ConfigProto(log_device_placement=False)
    with tf.Session(config=sessconfig) as sess:
        tf.initialize_all_variables().run()

        best_accuracy = (0.0, "iter: -1")
        best_f1_score = (0.0, "iter: -1")

        # Start training for each batch and loop epochs

        worst_batches = []

        for i in range(config.training_epochs):

            # Loop batches for an epoch:
            shuffled_X, shuffled_y = shuffle(X_train, y_train, random_state=i*42)
            for start, end in zip(range(0, config.train_count, config.batch_size),
                                  range(config.batch_size, config.train_count + 1, config.batch_size)):

                _, train_acc, train_loss, train_pred = sess.run(
                    [optimize, accuracy, loss, pred_y],
                    feed_dict={
                        X: shuffled_X[start:end],
                        Y: shuffled_y[start:end],
                        is_train: True
                    }
                )

                worst_batches.append(
                    (train_loss, shuffled_X[start:end], shuffled_y[start:end])
                )
                worst_batches = list(sorted(worst_batches))[-5:]  # Keep 5 poorest

            # Train F1 score is not on boosting
            train_f1_score = metrics.f1_score(
                shuffled_y[start:end].argmax(1), train_pred.argmax(1), average="weighted"
            )

            # Retrain on top worst batches of this epoch (boosting):
            # a.k.a. "focus on the hardest exercises while training":
            for _, x_, y_ in worst_batches:

                _, train_acc, train_loss, train_pred = sess.run(
                    [optimize, accuracy, loss, pred_y],
                    feed_dict={
                        X: x_,
                        Y: y_,
                        is_train: True
                    }
                )

            # Test completely at the end of every epoch:
            # Calculate accuracy and F1 score
            pred_out, accuracy_out, loss_out = sess.run(
                [pred_y, accuracy, loss],
                feed_dict={
                    X: X_test,
                    Y: y_test,
                    is_train: False
                }
            )

            # "y_test.argmax(1)": could be optimised by being computed once...
            f1_score_out = metrics.f1_score(
                y_test.argmax(1), pred_out.argmax(1), average="weighted"
            )

            print (
                "iter: {}, ".format(i) + \
                "train loss: {}, ".format(train_loss) + \
                "train accuracy: {}, ".format(train_acc) + \
                "train F1-score: {}, ".format(train_f1_score) + \
                "test loss: {}, ".format(loss_out) + \
                "test accuracy: {}, ".format(accuracy_out) + \
                "test F1-score: {}".format(f1_score_out)
            )

            best_accuracy = max(best_accuracy, (accuracy_out, "iter: {}".format(i)))
            best_f1_score = max(best_f1_score, (f1_score_out, "iter: {}".format(i)))

        print("")
        print("final test accuracy: {}".format(accuracy_out))
        print("best epoch's test accuracy: {}".format(best_accuracy))
        print("final F1 score: {}".format(f1_score_out))
        print("best epoch's F1 score: {}".format(best_f1_score))
        print("")

    # returning both final and bests accuracies and f1 scores.
    return accuracy_out, best_accuracy, f1_score_out, best_f1_score


In [23]:
one_hot(np.asarray([[3], [0], [3]]))

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

## Let's get serious and build the neural network:

In [47]:
n_layers_in_highway = 0
n_stacked_layers = 3
trial_name = "{}x{}".format(n_layers_in_highway, n_stacked_layers)

for learning_rate in [0.0025]:  # [0.01, 0.007, 0.001, 0.0007, 0.0001]:
    for lambda_loss_amount in [0.005]:
        for clip_gradients in [15.0]:
            print "learning_rate: {}".format(learning_rate)
            print "lambda_loss_amount: {}".format(lambda_loss_amount)
            print ""

            class EditedConfig(Config):
                def __init__(self, X, Y):
                    super(EditedConfig, self).__init__(X, Y)

                    # Edit only some parameters:
                    self.learning_rate = learning_rate
                    self.lambda_loss_amount = lambda_loss_amount
                    self.clip_gradients = clip_gradients
                    # Architecture params:
                    self.n_layers_in_highway = n_layers_in_highway
                    self.n_stacked_layers = n_stacked_layers

            # # Useful catch upon looping (e.g.: not enough memory)
            # try:
            #     accuracy_out, best_accuracy = run_with_config(EditedConfig)
            # except:
            #     accuracy_out, best_accuracy = -1, -1
            accuracy_out, best_accuracy, f1_score_out, best_f1_score = (
                run_with_config(EditedConfig, X_train, one_hot(y_train.astype(int)), X_test, one_hot(y_test.astype(int)))
            )
            print (accuracy_out, best_accuracy, f1_score_out, best_f1_score)

            with open('{}_result.txt'.format(trial_name), 'a') as f:
                f.write(str(learning_rate) + ' \t' + str(lambda_loss_amount) + ' \t' + str(clip_gradients) + ' \t' + str(
                    accuracy_out) + ' \t' + str(best_accuracy) + ' \t' + str(f1_score_out) + ' \t' + str(best_f1_score) + '\n\n')

            print "________________________________________________________"
        print ""
print "Done."

learning_rate: 0.0025
lambda_loss_amount: 0.005

Some useful info to get an insight on dataset's shape and normalisation:
features shape, labels shape, each features mean, each features standard deviation
((443, 1000, 22), (443, 4), 0.0016580587469994378, 0.997292271181712)
the dataset is therefore properly normalised, as expected.
(1000, ?, 22)
(?, 22)
(1000, '(?, 22)')

Creating hidden #1:
(1000, '(?, 10)')

Creating hidden #2:
(1000, '(?, 10)')

Creating hidden #3:
(1000, '(?, 10)')

Unregularised variables:
LSTM_network/layer_1/relu_fc_biases_noreg:0
LSTM_network/layer_2/relu_fc_biases_noreg:0
LSTM_network/layer_3/relu_fc_biases_noreg:0
LSTM_network/relu_fc_biases_noreg:0
iter: 0, train loss: 1.54455471039, train accuracy: 0.259999990463, train F1-score: 0.202718616182, test loss: 1.54643905163, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 1, train loss: 1.48272764683, train accuracy: 0.280000001192, train F1-score: 0.127151515152, test loss: 1.49025642872, t

iter: 42, train loss: 1.38564300537, train accuracy: 0.25, train F1-score: 0.0728925619835, test loss: 1.39237308502, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 43, train loss: 1.38887023926, train accuracy: 0.25, train F1-score: 0.0929032258065, test loss: 1.39172005653, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 44, train loss: 1.38643968105, train accuracy: 0.25, train F1-score: 0.130387596899, test loss: 1.39120364189, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 45, train loss: 1.38727521896, train accuracy: 0.25, train F1-score: 0.163759398496, test loss: 1.39164364338, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 46, train loss: 1.38671064377, train accuracy: 0.25, train F1-score: 0.0860162601626, test loss: 1.39157688618, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 47, train loss: 1.38759434223, train accuracy: 0.25, train F1-score: 0.0929032258065, test loss:

iter: 88, train loss: 1.38671553135, train accuracy: 0.25, train F1-score: 0.107301587302, test loss: 1.3919506073, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 89, train loss: 1.38658988476, train accuracy: 0.25, train F1-score: 0.172537313433, test loss: 1.39157533646, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 90, train loss: 1.38666558266, train accuracy: 0.25, train F1-score: 0.130387596899, test loss: 1.39189183712, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 91, train loss: 1.38670945168, train accuracy: 0.25, train F1-score: 0.0606722689076, test loss: 1.39204204082, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 92, train loss: 1.38666248322, train accuracy: 0.25, train F1-score: 0.1, test loss: 1.39167749882, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 93, train loss: 1.38665521145, train accuracy: 0.25, train F1-score: 0.0860162601626, test loss: 1.39173161983

iter: 135, train loss: 1.38675320148, train accuracy: 0.25, train F1-score: 0.146717557252, test loss: 1.39225935936, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 136, train loss: 1.38678014278, train accuracy: 0.25, train F1-score: 0.138461538462, test loss: 1.39242649078, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 137, train loss: 1.38668012619, train accuracy: 0.25, train F1-score: 0.1, test loss: 1.39214730263, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 138, train loss: 1.38675177097, train accuracy: 0.25, train F1-score: 0.0929032258065, test loss: 1.39230310917, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 139, train loss: 1.38673758507, train accuracy: 0.25, train F1-score: 0.0793442622951, test loss: 1.39230072498, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 140, train loss: 1.38675570488, train accuracy: 0.25, train F1-score: 0.0860162601626, test loss: 1.392

iter: 182, train loss: 1.38659787178, train accuracy: 0.25, train F1-score: 0.155151515152, test loss: 1.39203643799, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 183, train loss: 1.38666951656, train accuracy: 0.25, train F1-score: 0.0860162601626, test loss: 1.39205610752, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 184, train loss: 1.38665986061, train accuracy: 0.25, train F1-score: 0.1, test loss: 1.39203095436, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 185, train loss: 1.38671886921, train accuracy: 0.25, train F1-score: 0.0929032258065, test loss: 1.39229524136, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 186, train loss: 1.38680315018, train accuracy: 0.25, train F1-score: 0.163759398496, test loss: 1.39257228374, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 187, train loss: 1.38676559925, train accuracy: 0.25, train F1-score: 0.1, test loss: 1.39222836494, te

iter: 228, train loss: 1.38684093952, train accuracy: 0.25, train F1-score: 0.0494017094017, test loss: 1.39258515835, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 229, train loss: 1.38682246208, train accuracy: 0.25, train F1-score: 0.130387596899, test loss: 1.39243817329, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 230, train loss: 1.3868625164, train accuracy: 0.25, train F1-score: 0.0441379310345, test loss: 1.39270675182, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 231, train loss: 1.38675522804, train accuracy: 0.25, train F1-score: 0.1, test loss: 1.3923561573, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 232, train loss: 1.38673710823, train accuracy: 0.25, train F1-score: 0.138461538462, test loss: 1.39214289188, test accuracy: 0.203160271049, test F1-score: 0.0686094723423
iter: 233, train loss: 1.38665938377, train accuracy: 0.25, train F1-score: 0.146717557252, test loss: 1.391878

In [22]:
print(tf.__version__)

1.6.0


## Conclusion

Outstandingly, **the final accuracy is of 20.3%**! 

## References

The code is based on the following repository: 
> Guillaume Chevalier, LSTMs for Human Activity Recognition, 2016
> https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition
